In [1]:
!pip install dash


 Load and Explore the Dataset

In [ ]:
import pandas as pd


df = pd.read_csv('C:/Users/jagas/OneDrive/Desktop/c-19/country_wise_latest.csv')  # Example CSV filename


Extract basic information

In [3]:
print(df.head())


  Country/Region  Confirmed  Deaths  Recovered  Active  New cases  New deaths  \
0    Afghanistan      36263    1269      25198    9796        106          10   
1        Albania       4880     144       2745    1991        117           6   
2        Algeria      27973    1163      18837    7973        616           8   
3        Andorra        907      52        803      52         10           0   
4         Angola        950      41        242     667         18           1   

   New recovered  Deaths / 100 Cases  Recovered / 100 Cases  \
0             18                3.50                  69.49   
1             63                2.95                  56.25   
2            749                4.16                  67.34   
3              0                5.73                  88.53   
4              0                4.32                  25.47   

   Deaths / 100 Recovered  Confirmed last week  1 week change  \
0                    5.04                35526            737   
1   

In [4]:
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country/Region          187 non-null    object 
 1   Confirmed               187 non-null    int64  
 2   Deaths                  187 non-null    int64  
 3   Recovered               187 non-null    int64  
 4   Active                  187 non-null    int64  
 5   New cases               187 non-null    int64  
 6   New deaths              187 non-null    int64  
 7   New recovered           187 non-null    int64  
 8   Deaths / 100 Cases      187 non-null    float64
 9   Recovered / 100 Cases   187 non-null    float64
 10  Deaths / 100 Recovered  187 non-null    float64
 11  Confirmed last week     187 non-null    int64  
 12  1 week change           187 non-null    int64  
 13  1 week % increase       187 non-null    float64
 14  WHO Region              187 non-null    ob

 Create Visualizations

 Bar chart : Top 10 affected countries.

In [5]:
import plotly.express as px

top_countries = df.sort_values(by='Confirmed', ascending=False).head(10)
fig = px.bar(top_countries, x='Country/Region', y='Confirmed', title='Top 10 Countries by Confirmed Cases')
fig.show()


Pie Chart: Share of Global Deaths

In [9]:
import plotly.express as px

# Sort by deaths and select top contributors (e.g., top 10 countries)
top_deaths = df.sort_values(by='Deaths', ascending=False).head(10)

# Create pie chart
fig = px.pie(top_deaths,
             names='Country/Region',
             values='Deaths',
             title='Share of Global COVID-19 Deaths (Top 10 Countries)',
             hole=0.3)  # optional: to create a donut chart

fig.show()


Heat Maps: Case density by region.

In [6]:
fig = px.choropleth(df,
                    locations="Country/Region",
                    locationmode="country names",
                    color="Confirmed",
                    hover_name="Country/Region",
                    color_continuous_scale="Reds",
                    title="Global COVID-19 Confirmed Cases Heat Map")
fig.show()


Interactive Dashboard with Dash

In [8]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("COVID-19 Global Dashboard"),
    dcc.Graph(id='bar-chart'),
    dcc.Dropdown(
        id='metric-dropdown',
        options=[
            {'label': 'Confirmed', 'value': 'Confirmed'},
            {'label': 'Deaths', 'value': 'Deaths'},
            {'label': 'Recovered', 'value': 'Recovered'},
            {'label': 'Active', 'value': 'Active'},
        ],
        value='Confirmed'
    ),
    dcc.Graph(id='choropleth')
])

@app.callback(
    Output('bar-chart', 'figure'),
    Output('choropleth', 'figure'),
    Input('metric-dropdown', 'value')
)
def update_graphs(metric):
    top_countries = df.sort_values(by=metric, ascending=False).head(10)
    bar_fig = px.bar(top_countries, x='Country/Region', y=metric, title=f'Top 10 Countries by {metric}')
    
    choropleth_fig = px.choropleth(df,
                                    locations="Country/Region",
                                    locationmode="country names",
                                    color=metric,
                                    hover_name="Country/Region",
                                    color_continuous_scale="Viridis",
                                    title=f'Global COVID-19 {metric} Heat Map')
    return bar_fig, choropleth_fig

if __name__ == '__main__':
    app.run(debug=True)

